In [19]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [5]:
# load data CSV
def load_data():
    data = pd.read_csv('seeds_dataset.csv', names=['a','b','c','d','e','f','g','t'])
    features = data[['a','b','c','d','e','f','g']]
    target = data[['t']]
    
    return features, target

In [6]:
# Pre-processing data
def preprocess_data(features, target):
    features = MinMaxScaler().fit_transform(features)
    target = OneHotEncoder(sparse=False).fit_transform(target)
    
    return features, target

In [7]:
# Inisialisasi layers, weights, dan bias
layers = {
    'input' : 7 ,
    'hidden' : 7 , # 2 / 3 dari input + outputnya
    'output' : 3,
}

weights = {
    'hidden' : tf.Variable( tf.random.normal( [layers['input'], layers['hidden']] ) ),
    'output' : tf.Variable( tf.random.normal( [layers['hidden'], layers['output']] ) ),
}

bias = {
    'hidden' : tf.Variable( tf.random.normal( [layers['hidden'] ])),
    'output' : tf.Variable( tf.random.normal( [layers['output'] ])),
}

In [8]:
def activate(y):
    return tf.nn.sigmoid(y)

def feed_forward(features):
    Wx1b = tf.matmul(features, weights['hidden']) + bias['hidden']
    y1 = activate(Wx1b)

    Wx2b = tf.matmul(y1, weights['output']) + bias['output']
    y2 = activate(Wx2b)

    return y2

In [9]:
features_container = tf.placeholder(tf.float32, [None, layers['input']])
target_container = tf.placeholder(tf.float32, [None, layers['output']]) #(?, 3)

predicted_target = feed_forward(features_container)
error = tf.reduce_mean(0.5 * (target_container - predicted_target)**2 )

learning_rate = 0.2
epoch = 5000

train = tf.train.GradientDescentOptimizer(learning_rate).minimize(error)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
# Data Processing
features, target = load_data()
features, target = preprocess_data(features, target)

feature_train, feature_test, target_train, target_test = train_test_split(features, target, test_size=0.2)

         a      b       c      d      e      f      g
0    15.26  14.84  0.8710  5.763  3.312  2.221  5.220
1    14.88  14.57  0.8811  5.554  3.333  1.018  4.956
2    14.29  14.09  0.9050  5.291  3.337  2.699  4.825
3    13.84  13.94  0.8955  5.324  3.379  2.259  4.805
4    16.14  14.99  0.9034  5.658  3.562  1.355  5.175
..     ...    ...     ...    ...    ...    ...    ...
205  12.19  13.20  0.8783  5.137  2.981  3.631  4.870
206  11.23  12.88  0.8511  5.140  2.795  4.325  5.003
207  13.20  13.66  0.8883  5.236  3.232  8.315  5.056
208  11.84  13.21  0.8521  5.175  2.836  3.598  5.044
209  12.30  13.34  0.8684  5.243  2.974  5.637  5.063

[210 rows x 7 columns]
     t
0    1
1    1
2    1
3    1
4    1
..  ..
205  3
206  3
207  3
208  3
209  3

[210 rows x 1 columns]


In [21]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(epoch):
        train_data = {
            features_container: feature_train,
            target_container: target_train,
        }
        sess.run(train, feed_dict = train_data)
        loss = sess.run(error, feed_dict = train_data)

        if i % 500 ==  0:
            print(f'Epoch {i}, Loss Rate: {loss}')

    test_data = {
        features_container: feature_test,
        target_container: target_test,
    }

    accuracy = tf.equal(tf.argmax(target_container, axis = 1), tf.argmax(predicted_target, axis = 1) )
    print(f'Accuracy Row : {sess.run(accuracy, feed_dict = test_data)}')
    accuracy = tf.reduce_mean(tf.cast(accuracy, tf.float32))
    print(f'Final Accuracy : {sess.run(accuracy, feed_dict = test_data) * 100}%')
    conf = tf.math.confusion_matrix(tf.argmax(target_container, axis = 1), tf.argmax(predicted_target, axis = 1) )
    print('Confusion Matrix:')
    print(sess.run(conf, feed_dict = test_data))

Epoch 0, Loss Rate: 0.2001960277557373
Epoch 500, Loss Rate: 0.09579156339168549
Epoch 1000, Loss Rate: 0.07722152024507523
Epoch 1500, Loss Rate: 0.06430569291114807
Epoch 2000, Loss Rate: 0.05583174154162407
Epoch 2500, Loss Rate: 0.049832168966531754
Epoch 3000, Loss Rate: 0.0451793372631073
Epoch 3500, Loss Rate: 0.04136084392666817
Epoch 4000, Loss Rate: 0.038138069212436676
Epoch 4500, Loss Rate: 0.035384420305490494
Accuracy Row : [ True  True  True  True False  True  True False  True  True  True  True
 False  True  True  True False  True  True  True  True  True  True  True
  True  True  True  True  True  True False  True  True  True False  True
  True  True  True  True  True  True]
Final Accuracy : 85.71428656578064%
Confusion Matrix:
[[14  2  2]
 [ 1  9  0]
 [ 1  0 13]]
